In [1]:
## Check if there is a GPU
!nvidia-smi

Fri Apr 14 13:06:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 517.66       Driver Version: 517.66       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   58C    P0    15W /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# Check that PyTorch sees it
import torch
import torch.cuda

print(f"Is CUDA available: {torch.cuda.is_available()}")
# True
#print(f"CUDA device: {torch.cuda.get_device_name(torch.cuda.current_device())}")
# Tesla T4

Is CUDA available: False


In [11]:
## Training SentenceTransformers model from scratch
%time
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

path = "sample-logs.txt"

## Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

## Customize training
tokenizer.train(files=[path], vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 14.5 µs





In [12]:
## Salva modelo

!mkdir LogFiles
tokenizer.save_model("LogFiles")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
mkdir: cannot create directory ‘LogFiles’: File exists


['LogFiles/vocab.json', 'LogFiles/merges.txt']

In [13]:
## Testa modelo

from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./LogFiles/vocab.json",
    "./LogFiles/merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

print(tokenizer.encode("ERROR MEMORY"))
print(tokenizer.encode("ERROR MEMORY").tokens)

Encoding(num_tokens=10, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['<s>', 'ER', 'RO', 'R', 'ĠM', 'E', 'M', 'O', 'RY', '</s>']


In [4]:
## Check that we have a GPU
#!nvidia-smi

# Check that PyTorch sees it
import torch
torch.cuda.is_available()

False

In [14]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [15]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./LogFiles", max_len=512)

In [16]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
model.num_parameters()

83504416

In [17]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=path,
    block_size=128,
)

/home/vbertalan/anaconda3/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [18]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [20]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./LogFiles",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [21]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/home/vbertalan/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 63
  Number of trainable parameters = 83504416


  0%|          | 0/63 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 321.8437, 'train_samples_per_second': 12.428, 'train_steps_per_second': 0.196, 'train_loss': 7.261244032118055, 'epoch': 1.0}
CPU times: user 36min 29s, sys: 3min 37s, total: 40min 6s
Wall time: 5min 21s


TrainOutput(global_step=63, training_loss=7.261244032118055, metrics={'train_runtime': 321.8437, 'train_samples_per_second': 12.428, 'train_steps_per_second': 0.196, 'train_loss': 7.261244032118055, 'epoch': 1.0})

In [22]:
trainer.save_model("./LogFiles")

Saving model checkpoint to ./LogFiles
Configuration saved in ./LogFiles/config.json
Model weights saved in ./LogFiles/pytorch_model.bin


In [26]:
#from sentence_transformers import SentenceTransformer
#model = SentenceTransformer('./LogFiles')
#frase_teste = "ERROR LINE BREAK"
#resultado = model.encode(frase_teste)